# [비디오를 이용한 행동 분류](https://www.kaggle.com/c/2021-ml-tp4/overview/description)
> 2021학년도 1학기 기계학습 텀프로젝트 \#5
## 참고
- [2D 데이터 분류 문제](https://www.kaggle.com/c/2021-ml-p12)
- [SIFT 특징량](https://bskyvision.com/21)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp4/sample_submission.csv
/kaggle/input/2021-ml-tp4/class_info.csv
/kaggle/input/2021-ml-tp4/train_label.csv
/kaggle/input/2021-ml-tp4/test/0261.avi
/kaggle/input/2021-ml-tp4/test/0435.avi
/kaggle/input/2021-ml-tp4/test/0309.avi
/kaggle/input/2021-ml-tp4/test/0039.avi
/kaggle/input/2021-ml-tp4/test/0162.avi
/kaggle/input/2021-ml-tp4/test/0402.avi
/kaggle/input/2021-ml-tp4/test/0255.avi
/kaggle/input/2021-ml-tp4/test/0416.avi
/kaggle/input/2021-ml-tp4/test/0354.avi
/kaggle/input/2021-ml-tp4/test/0486.avi
/kaggle/input/2021-ml-tp4/test/0113.avi
/kaggle/input/2021-ml-tp4/test/0106.avi
/kaggle/input/2021-ml-tp4/test/0291.avi
/kaggle/input/2021-ml-tp4/test/0407.avi
/kaggle/input/2021-ml-tp4/test/0247.avi
/kaggle/input/2021-ml-tp4/test/0442.avi
/kaggle/input/2021-ml-tp4/test/0193.avi
/kaggle/input/2021-ml-tp4/test/0044.avi
/kaggle/input/2021-ml-tp4/test/0136.avi
/kaggle/input/2021-ml-tp4/test/0332.avi
/kaggle/input/2021-ml-tp4/test/0311.avi
/kaggle/input/2021-ml-tp4/tes

In [2]:
# # SIFT 사용을 위해 버전을 맞춘다.
# ! yes | pip3 uninstall opencv-python
# ! yes | pip3 uninstall opencv-contrib-python
# ! yes | pip3 install opencv-python==3.4.2.17
# ! yes | pip3 install opencv-contrib-python==3.4.2.17
# cv2.__version__

In [3]:
import os
import numpy as np
import pandas as pd
import tqdm
import time
import cv2
import pickle
import torch

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from scipy.stats import mode
from sklearn.decomposition import PCA

from sklearn.model_selection import cross_validate, GridSearchCV

In [4]:
# 베이스라인 달성을 위한 파라미터 제공
arg_img_size = (128, 128)
arg_dense_sift = True
args_local_cluster = 200
args_global_cluster = 200
num_frame = 5
args_aggr = "vlad" # or "bow"
pca_vlad = 128

# 비디오 전처리 및 프레임 별 특징점(visual word) 추출 (Empty Module 1)

In [5]:
# train 비디오의 행동 분류 label read
root = "/kaggle/input/2021-ml-tp4/"
train_csv = os.path.join(root, "train_label.csv")
train_csv = pd.read_csv(train_csv)
train_csv_arr = np.asarray(train_csv)

# 데이터 셋에 존재하는 행동 분류 정보 read
classinfo = os.path.join(root, "class_info.csv")
classinfo = pd.read_csv(classinfo)
classinfo_arr = np.asarray(classinfo)


train_path = os.path.join(root, "train")
test_path = os.path.join(root, "test")

# train 비디오 경로
train_list = os.listdir(train_path)
train_list.sort()
train_list = [os.path.join(train_path, i) for i in train_list]

# test 비디오 경로
test_list = os.listdir(test_path)
test_list.sort()
test_list = [os.path.join(test_path, i) for i in test_list]

In [6]:
train_csv.head()

,Id,Category
0,0,BalanceBeam
1,1,BalanceBeam
2,2,JumpingJack
3,3,Bowling
4,4,BalanceBeam


In [7]:
classinfo.head()

,ClassId,Category
0,0,ApplyEyeMakeup
1,1,ApplyLipstick
2,2,Archery
3,3,BabyCrawling
4,4,BalanceBeam


In [8]:
def video_to_frame(video_path, size, num_frame):
    
    #########################################################
    ## 비디오에서 프레임을 추출해주는 함수
    ## 
    ## Input 
    ##     video_path : 한 비디오의 경로
    ##     size : 비디오 내의 프레임을 읽을 때, 원하는 해상도 크기
    ##     num_frames : 한 비디오 내에서 읽을 프레임의 수
    ##
    ## Output
    ##     frames : 읽고 저장한 총 프레임
    #########################################################
    
    cap = cv2.VideoCapture(video_path)
    
    # 한 비디오의 총 프레임 수 반환 및 원하는 프레임 수 많큼 읽기 위해 읽을 프레임의 인덱스 설정
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sel_ind = np.linspace(0, total_frame-1, num_frame).astype("int")
    
    
    num=0
    frames = []
    for i in range(total_frame):
        
        # 읽을 프레임 인덱스의 경우 프레임 읽어 메모리에 저장, 아닐 경우 지나감
        if i in sel_ind:
            res, frame = cap.read()
            # 원하는 해상도로 조절 및 grayscale로 변환
            frame = cv2.resize(frame, size, interpolation = cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(frame)
        else:
            res = cap.grab()        
    cap.release()
    frames = np.asarray(frames)

    return frames

In [9]:
def computeSIFT(data, dense=False):
    
    #########################################################
    ## 비디오 내의 프레임에서 특징점(visual word)을 SIFT or DenseSIFT로 추출해주는 함수
    ## 
    ## Input 
    ##     data : 한 비디오에서 읽고 저장한 프레임
    ##     dense : SIFT or DenseSIFT 사용 여부
    ##
    ## Output
    ##     x : 프레임에 대해 추출된 특징점(visual word), dict 형태 -> x[0]이면 0번째 인덱스 프레임의 특징점(visual word) [n,128] 확인 가능
    #########################################################
    
    x = {}
    for i in range(0, len(data)): 
        if dense:
            img = data[i]
            step_size = 8
            kp = [cv2.KeyPoint(x, y, step_size) 
                  for x in range(0, img.shape[0], step_size) 
                      for y in range(0, img.shape[1], step_size)] 
            
            ####### Empty Module 1 : DenseSIFT ########
            # 기본 SIFT 와 동일하게 정의 
            # 기본 SIFT 에서는 detectAndCompute를 사용했지만 
            # Dense SIFT는 위에서 생성한 keypoint를 사용해 compute 만을 진행 
            ###########################################
            ##
            sift = cv2.SIFT_create()
            kp, desc = sift.compute(img, kp)
            
        else:
            sift = cv2.SIFT_create()
            img = data[i]
            kp, desc = sift.detectAndCompute(img, None)
        x.update({i : desc})

    return x

In [10]:
# train 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 train_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
train_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(train_list, desc="Extract {} in train data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    train_local_desc.update({vid_path : local_desc})

# test 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 test_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
test_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(test_list, desc="Extract {} in test data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    test_local_desc.update({vid_path : local_desc})


Extract dsift in test data: 100%|██████████| 505/505 [00:30<00:00, 16.41it/s]


In [11]:
len(train_local_desc)

2020

In [12]:
# vedio_to_frame() 의 num_fram = 5, size = (128, 128)
curr_frame.shape

(5, 128, 128)

In [13]:
len(train_local_desc['/kaggle/input/2021-ml-tp4/train/0000.avi'])

5

In [14]:
# train_local_desc dict 안에 dict가 들어있는 것
train_local_desc['/kaggle/input/2021-ml-tp4/train/0000.avi'].keys()

dict_keys([0, 1, 2, 3, 4])

In [15]:
train_local_desc['/kaggle/input/2021-ml-tp4/train/0000.avi'][0].shape

(256, 128)

In [16]:
# train_local_desc_total.extend(vv) (아래 코드 확인) 
l = []

array = np.array([[1, 2, 3],
                  [4, 5, 6]])

l.extend(array)
print(array.shape)
print(len(l))
print(l)

(2, 3)
2
[array([1, 2, 3]), array([4, 5, 6])]


In [17]:
print("\n\nAggregate SIFT descriptor")
start = time.time()


# train 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, -> n은 visual word의 개수?
# 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 train_frame_total에서 확인 가능 및 
# 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 train_local_info에서 확인 가능
train_frame_total = []
train_local_desc_total = []
train_local_info = {}
for k, v in train_local_desc.items():
    for kk, vv in v.items():
        ## kk = 0 ~ 4 
        l_num = 0
        if vv is not None:
            ## 첫번째 vv.shape = (256, 128) 이므로 256개가 추가 된다.
            train_local_desc_total.extend(vv)
            ## visual word들의 정보(비디오 내의 몇 번째 프레임에서 나온 특징점인지)
            train_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        ## 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 train_local_info에서 확인 가능
        train_local_info.update({k+", "+str(kk) : l_num})
train_local_desc_total = np.asarray(train_local_desc_total)
train_frame_total = np.asarray(train_frame_total)


# test 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 test_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 test_local_info에서 확인 가능
test_frame_total = []
test_local_desc_total = []
test_local_info = {}
for k, v in test_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            test_local_desc_total.extend(vv)
            test_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        test_local_info.update({k+", "+str(kk) : l_num})
test_local_desc_total = np.asarray(test_local_desc_total)
test_frame_total = np.asarray(test_frame_total)


print("\t{:3.2f}s\n\n".format(time.time()-start))



Aggregate SIFT descriptor
	3.31s




In [18]:
# 2020(vedio 수) * 5(frame 수) * 256(visual word 수) = 2585600 -> 모든 visual word
train_local_desc_total.shape

(2585600, 128)

In [19]:
train_frame_total.shape

(2585600,)

In [20]:
# vedio 마다 5개 frame 추출, frame 마다 256개의 visual word
# img_size = (128, 128), step_size = 8 이므로 16*16 = 256
train_frame_total[:256*10:256]

array(['/kaggle/input/2021-ml-tp4/train/0000.avi, 0',
       '/kaggle/input/2021-ml-tp4/train/0000.avi, 1',
       '/kaggle/input/2021-ml-tp4/train/0000.avi, 2',
       '/kaggle/input/2021-ml-tp4/train/0000.avi, 3',
       '/kaggle/input/2021-ml-tp4/train/0000.avi, 4',
       '/kaggle/input/2021-ml-tp4/train/0001.avi, 0',
       '/kaggle/input/2021-ml-tp4/train/0001.avi, 1',
       '/kaggle/input/2021-ml-tp4/train/0001.avi, 2',
       '/kaggle/input/2021-ml-tp4/train/0001.avi, 3',
       '/kaggle/input/2021-ml-tp4/train/0001.avi, 4'], dtype='<U43')

In [21]:
# 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 train_local_info에서 확인 가능
train_local_info['/kaggle/input/2021-ml-tp4/train/0000.avi, 0']

256

# 비디오 feature 기술을 위한 프레임 feature 기술 (Empty Module 2,3,4)

## try
- kmeans parameter tuning
- data decomposition -> pca -> 아래에 있음

In [22]:
## codebook
def clustering(train_desc, test_desc=None, n_clusters=200):
    #########################################################
    ## 모든 특징점들 중, 대표 특징점(codebook)을 선정하는 함수
    ## 
    ## Input 
    ##     train_desc : 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     test_desc : 모든 test 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     n_clusters : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     train_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     test_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     clusters : 대표 특징점(codebook)
    ##     kmeans : kmeans 인스턴스
    #########################################################
    
    
    
    ##### Empty Module 2 : 대표 특징점(codebook) 선정 ######
    # 제약 조건 : MiniBatchKMeans 사용, random_state=0 고정
    # sklearn의 MiniBatchKMeans를 활용하여 n_clusters 크기 만큼의 대표 특징점(codebook)을 선정
    ###########################################
    
    kmeans = MiniBatchKMeans(n_clusters=n_clusters,
                            random_state=0)\
                            .fit(train_desc)
    train_pred = kmeans.predict(train_desc)
    clusters = kmeans.cluster_centers_
    
    if test_desc is not None:
        test_pred = kmeans.predict(test_desc)
    else:
        test_pred = None
    return train_pred, test_pred, clusters, kmeans

In [23]:
# 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들로 대표 특징점(codebook)을 만들고,
# train 비디오의 모든 프레임에서 추출된 특징점(visual word)과 test 비디오의 모든 프레임에서 추출된 
# 특징점(visual word)을 대표 특징점(codebook)에 할당
train_local_alloc, test_local_alloc, local_codebook, local_kmeans =\
    clustering(train_local_desc_total, test_local_desc_total, args_local_cluster)

In [24]:
train_local_alloc.shape  # alloc

(2585600,)

In [25]:
train_local_alloc[:10]

array([152,  96, 134,  84,  27,  27,  42,  42,  42,  42], dtype=int32)

In [26]:
# centers
# n_clusters = 200 이므로 
local_codebook.shape  

(200, 128)

---
VLAD

In [27]:
def VLAD(X, alloc, centers):
    #########################################################
    ## 이미지 feature인 VLAD feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     X : 한 프레임의 특징점(visual word)들
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     centers : 대표 특징점(codebook)
    ##
    ## Output
    ##     V : VLAD feature
    #########################################################
    
    m,d = X.shape # (256, 128)
    k = centers.shape[0]  # centers.shape == (200, 128)
    
    # VLAD feature를 담기 위한 변수
    V=np.zeros([k,d])

    for i in range(k):
        if np.sum(alloc==i)>0:
            ####################### Empty Module 3 : VLAD ########################
            # 이미지에서 추출된 특징점(visual word) X와 이들이 대표 특징점(codebook)으로 할당된 정보 alloc을 이용해,
            # 동일한 대표 특징점(codebook)으로 할당된 특징점(visual word)들의 벡터 합 계산해 V[i]에 저장
            # hint : 바로 위 조건문의 조건 "alloc==i"의 의미를 파악하고 인덱싱에 활용
            ######################################################################
            
            V[i, :] = np.sum(X[alloc==i, :] - centers[i], axis=0)
    
    # 후처리 과정
    V = V.flatten()
    V = np.sign(V)*np.sqrt(np.abs(V))
    if np.sqrt(np.dot(V,V))!=0:
        V = V/np.sqrt(np.dot(V,V))
    return V


def BoW(alloc, n_cluster):
    #########################################################
    ## 이미지 feature인 BoW feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     n_cluster : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     V : BoW feature
    #########################################################
    
    ######################### Empty Module 4 : BoW ##########################
    # 이미지에서 추출된 특징점(visual word)이 대표 특징점(codebook)으로 할당된 정보 alloc의 histogram을 계산
    # np.histogram 함수 참고
    #########################################################################
    
    hist, bin_edges = np.histogram(alloc, bins=range(n_cluster + 1), normed=True)
    V = hist.flatten()
    
    return V

In [28]:
print("\n\nAllocate center & Descript local histogram")
start = time.time()

# Train 비디오 내의 프레임 별로 이미지 feature 기술 -> train_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> train_global_desc_key
train_global_desc = []
train_global_desc_key = []
vi=0
## v = 256
for k, v in train_local_info.items():
    if v!=0:
        if args_aggr=="bow":            
            hist_desc = BoW(train_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(train_local_desc_total[vi:vi+v],
                             train_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        train_global_desc.append(hist_desc)
        train_global_desc_key.append(k)
train_global_desc = np.asarray(train_global_desc)
train_global_desc_key = np.asarray(train_global_desc_key)


# Test 비디오 내의 프레임 별로 이미지 feature 기술 -> test_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> test_global_desc_key
test_global_desc = []
test_global_desc_key = []
vi=0
for k, v in test_local_info.items():
    if v!=0:
        if args_aggr=="bow":
            hist_desc = BoW(test_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(test_local_desc_total[vi:vi+v], test_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        test_global_desc.append(hist_desc)
        test_global_desc_key.append(k)
test_global_desc = np.asarray(test_global_desc)
test_global_desc_key = np.asarray(test_global_desc_key)

print("\t{:3.2f}s\n\n".format(time.time()-start))




Allocate center & Descript local histogram
	67.23s




In [29]:
train_local_desc_total[vi:vi+v].shape

(256, 128)

In [30]:
np.sum(train_local_desc_total[vi:vi+v], axis=0).shape

(128,)

In [31]:
hist_desc.shape

(25600,)

In [32]:
# 2020(vedio) * 5(frame)
train_global_desc.shape

(10100, 25600)

In [33]:
train_global_desc_key.shape

(10100,)

In [34]:
train_global_desc_key[:5*5:5]

array(['/kaggle/input/2021-ml-tp4/train/0000.avi, 0',
       '/kaggle/input/2021-ml-tp4/train/0001.avi, 0',
       '/kaggle/input/2021-ml-tp4/train/0002.avi, 0',
       '/kaggle/input/2021-ml-tp4/train/0003.avi, 0',
       '/kaggle/input/2021-ml-tp4/train/0004.avi, 0'], dtype='<U43')

In [35]:
# VLAD feature의 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_global_desc)
    train_global_desc = pca.transform(train_global_desc)
    test_global_desc = pca.transform(test_global_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))



Reduce dim of descriptor of the frames with PCA
	36.96s




In [36]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 train 비디오의 각 프레임 별 label 가공
train_global_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in train_global_desc_key])
train_global_label = []
for fid in train_global_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_global_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_global_label = np.asarray(train_global_label).ravel()

# 분류를 위해, 행동 분류에 대한 test 비디오의 각 프레임 별 id 가공 
test_global_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in test_global_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))



Processing label
	0.47s




In [37]:
train_global_id.shape

(10100,)

In [38]:
train_global_id[:5*5]

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4])

In [39]:
np.unique(train_global_id)

array([   0,    1,    2, ..., 2017, 2018, 2019])

In [40]:
train_global_label[:50]

array([ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4, 47, 47, 47, 47, 47, 15, 15,
       15, 15, 15,  4,  4,  4,  4,  4, 68, 68, 68, 68, 68,  9,  9,  9,  9,
        9, 38, 38, 38, 38, 38, 19, 19, 19, 19, 19, 24, 24, 24, 24, 24])

In [41]:
def saveFile(predict, predict_id, name, best_params=None):
    #########################################################
    ## 결과를 저장하기 위한 함수
    ## 
    ## Input 
    ##     predict : 모든 test 비디오의 행동 예측 값
    ##     predict_id : 모든 test 비디오의 id
    ##     name : 원하는 저장 파일 이름
    ##     best_params : 원하는 instance 저장 시 사용
    ##
    #########################################################
    
    data = np.concatenate((np.expand_dims(predict_id.astype("str"), axis=1),
                           np.expand_dims(predict.astype("str"), axis=1)), axis=1)
    csv = pd.DataFrame(data, columns=['Id', 'Category'])
    csv.to_csv(name + ".csv", index=False)
    
    if best_params:
        f = open(name + ".pickle", "wb")
        pickle.dump(best_params, f, 2)

# 비디오의 모든 프레임에서 얻은 feature를 평균내어 비디오 feature 생성 및 분류 (Empty Module 5)

In [42]:
classinfo_arr

array([[0, 'ApplyEyeMakeup'],
       [1, 'ApplyLipstick'],
       [2, 'Archery'],
       [3, 'BabyCrawling'],
       [4, 'BalanceBeam'],
       [5, 'BandMarching'],
       [6, 'BaseballPitch'],
       [7, 'Basketball'],
       [8, 'BasketballDunk'],
       [9, 'BenchPress'],
       [10, 'Biking'],
       [11, 'Billiards'],
       [12, 'BlowDryHair'],
       [13, 'BlowingCandles'],
       [14, 'BodyWeightSquats'],
       [15, 'Bowling'],
       [16, 'BoxingPunchingBag'],
       [17, 'BoxingSpeedBag'],
       [18, 'BreastStroke'],
       [19, 'BrushingTeeth'],
       [20, 'CleanAndJerk'],
       [21, 'CliffDiving'],
       [22, 'CricketBowling'],
       [23, 'CricketShot'],
       [24, 'CuttingInKitchen'],
       [25, 'Diving'],
       [26, 'Drumming'],
       [27, 'Fencing'],
       [28, 'FieldHockeyPenalty'],
       [29, 'FloorGymnastics'],
       [30, 'FrisbeeCatch'],
       [31, 'FrontCrawl'],
       [32, 'GolfSwing'],
       [33, 'Haircut'],
       [34, 'HammerThrow'],
       [35, '

In [43]:
print("\n\nSVM global averaging in frame")
start = time.time()
#################### Empty Module 6 : SVM (averaging) #####################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 비디오 별로 평균 내어 비디오 feature로 사용
# 비디오 feature로 학습 후, 행동 예측
# hint : 위 셀에서 선언한 train_global_id, train_global_label, test_global_id 활용
###########################################################################

###########################
X_train = []
y_train = []
for i in range(0, train_global_id.shape[0], num_frame):
    X_train.append(np.mean(train_global_desc[i:i+num_frame, :], axis=0))
    y_train.append(train_global_label[i])
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_test = []
for i in range(0, len(test_global_desc), num_frame):
    X_test.append(np.mean(test_global_desc[i:i+num_frame, :], axis=0))  
X_test = np.asarray(X_test)
    

svm = SVC(random_state=0)
cv_results = cross_validate(svm, X_train, y_train,
                            return_train_score=True)
svm.fit(X_train, y_train)
svm_predict = svm.predict(X_test)
###########################

saveFile(classinfo_arr[svm_predict][:,1],
         np.arange(len(test_list)), "svm_global_averaging")
print("\t{:3.2f}s\n\n".format(time.time()-start))
print(f"train: {np.mean(cv_results['train_score'])}, "+\
      f"test: {np.mean(cv_results['test_score'])}")



SVM global averaging in frame
	8.99s


train: 0.9497524752475247, test: 0.2816831683168317


In [44]:
# train이 test에 비해 매우 높아 overfitting이 의심된다...

param_grid = {
    'C': [1, 10, 100, 1000, 10000],
    'tol': [1e-3, 1e-2, 1e-1, 1e0]
}

gridsearch = GridSearchCV(SVC(random_state=0),
                        param_grid, return_train_score=True)
gridsearch.fit(X_train, y_train)
svm_predict = gridsearch.predict(X_test)
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)),
         "gridsearch_global_averaging")
best_params = gridsearch.best_params_
print(best_params)
pd.DataFrame(gridsearch.cv_results_)\
    .sort_values(by='mean_test_score',
                 ascending=False).loc[['mean_test_score', 'mean_train_score']].head(5)

{'C': 10, 'tol': 0.001}


KeyError: "None of [Index(['mean_test_score', 'mean_train_score'], dtype='object')] are in the [index]"

In [45]:
pd.DataFrame(gridsearch.cv_results_)\
    .sort_values(by='mean_test_score',
                 ascending=False)[['mean_test_score', 'mean_train_score']].head()

,mean_test_score,mean_train_score
4,0.304455,1.0
16,0.304455,1.0
8,0.304455,1.0
12,0.304455,1.0
13,0.303960,1.0


In [46]:
!apt-get install build-essential swig -y
!pip install auto-sklearn




build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 23 not upgraded.
Need to get 1100 kB of archives.
After this operation, 5822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6460 B]
Fetched 1100 kB in 2s (461 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package swig3.0.
(Reading database ... 95165 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpacking swig (3.0.12-1) ...
Setting

In [47]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [48]:
from autosklearn.pipeline.components import classification
print(classification._classifiers.keys())
estimators = ['libsvm_svc']

odict_keys(['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'mlp', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd'])


In [49]:
import autosklearn.classification
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    per_run_time_limit=60,
    include_preprocessors = None,
    include_estimators = estimators,
    n_jobs=-1
)
automl.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


AutoSklearnClassifier(include_estimators=['libsvm_svc'], n_jobs=-1,
                      per_run_time_limit=60, time_left_for_this_task=300)

In [50]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 3b382bee-c75e-11eb-8014-0242ac130202
  Metric: accuracy
  Best validation score: 0.275862
  Number of target algorithm runs: 82
  Number of successful target algorithm runs: 76
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 2



In [51]:
svm_predict = automl.predict(X_test)
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)),
         "automl_global_averaging")

# 비디오의 모든 프레임에서 얻은 feature로 분류기를 사용해 예측하고 예측치 중 가장 많은 빈도를 나타낸 행동을 선정 (Empty Module 6)

In [52]:
mode(np.array([0, 0, 1, 1, 1, 1])).mode

array([1])

In [53]:
print("\n\nSVM global voting in frame")
start = time.time()
############################### Empty Module 5 : SVM (voting) ##################################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 모두 사용해 SVM 학습
# 학습 시, 각 이미지 feature의 label은 해당되는 비디오의 label(행동)로 사용
# 프레임 별로 행동 예측 후, 같은 비디오 내 프레임의 행동 예측 값의 최빈값을 해당 비디오의 행동 예측 값으로 선정
# hint : 위 셀에서 선언한 train_global_label, test_global_id 및 mode 활용
################################################################################################

###########################
X_train = train_global_desc.copy()
y_train = train_global_label.copy()

X_test = test_global_desc.copy()

svm = SVC(random_state=0)
svm.fit(X_train, y_train)
svm_predict = svm.predict(X_test)
svm_predict = [mode(svm_predict[i:i+5]).mode[0] 
                   for i in range(0, len(svm_predict), 5)]
###########################

saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_voting")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global voting in frame
	25.28s




In [54]:
# Empty Module 6 에서의 best params 를 이용

print("\n\nSVM global voting in frame")
start = time.time()

X_train = train_global_desc.copy()
y_train = train_global_label.copy()

X_test = test_global_desc.copy()

svm = SVC(random_state=0)
svm.set_params(**best_params)
svm.fit(X_train, y_train)
svm_predict = svm.predict(X_test)
svm_predict = [mode(svm_predict[i:i+5]).mode[0] 
                   for i in range(0, len(svm_predict), 5)]


saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)),
         "best_params_svm_global_voting")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global voting in frame
	26.09s




# 프레임 feature에서 대표되는 feature를 선정 후 BoW or VLAD 방식으로 비디오 feature를 기술 (Empty Module 7)

In [55]:
train_global_alloc, test_global_alloc, global_codebook, global_kmeans =\
    clustering(train_global_desc, test_global_desc, args_global_cluster)

In [56]:
train_global_alloc.shape

(10100,)

In [57]:
print("\n\nAllocate center & Descript global histogram")
start = time.time()
train_vid_names = np.asarray([i.split(", ")[0] for i in train_global_desc_key])
train_vid_names_u = np.unique(train_vid_names)

# Train 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
train_video_desc = []
train_video_desc_key = []
for vid_name in train_vid_names_u:
    cind = np.where(vid_name==train_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(train_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(train_global_desc[cind], train_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    train_video_desc.append(hist_desc)
    train_video_desc_key.append(vid_name)
train_video_desc = np.asarray(train_video_desc)
train_video_desc_key = np.asarray(train_video_desc_key)

# Test 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
test_vid_names = np.asarray([i.split(", ")[0] for i in test_global_desc_key])
test_vid_names_u = np.unique(test_vid_names)

test_video_desc = []
test_video_desc_key = []
for vid_name in test_vid_names_u:
    cind = np.where(vid_name==test_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(test_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(test_global_desc[cind], test_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    test_video_desc.append(hist_desc)
    test_video_desc_key.append(vid_name)
test_video_desc = np.asarray(test_video_desc)
test_video_desc_key = np.asarray(test_video_desc_key)


print("\t{:3.2f}s\n\n".format(time.time()-start))




Allocate center & Descript global histogram
	10.78s




In [58]:
train_video_desc.shape

(2020, 25600)

In [59]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 각 train 비디오 별 label 가공
train_video_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in train_video_desc_key])
train_video_label = []
for fid in train_video_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_video_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_video_label = np.asarray(train_video_label).ravel()

# 분류를 위해, 행동 분류에 대한 각 test 비디오 별 id 가공
test_video_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in test_video_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))



Processing label
	0.10s




In [60]:
# 이미지 feature에 대해 다시 한번 VLAD feature 기술 방식을 사용하여 video feature를 기술한 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소

if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_video_desc)
    train_video_desc = pca.transform(train_video_desc)
    test_video_desc = pca.transform(test_video_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))



Reduce dim of descriptor of the frames with PCA
	8.61s




In [61]:
print("\n\nSVM video descriptor")
start = time.time()
######################## Empty Module 7 : SVM (video feature) #########################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 이전 이미지 feature를 기반으로 각 기술방식(BoW, VLAD)을 한번 더 적용해 만든 비디오 feature 사용
# 비디오 feature로 학습 후, 행동 예측
#######################################################################################

###########################
X_train = train_video_desc.copy()
y_train = train_video_label.copy()

X_test = test_video_desc.copy()

svm = SVC(random_state=0)
cv_results = cross_validate(svm, X_train, y_train,
                            return_train_score=True)
svm.fit(X_train, y_train)
svm_predict = svm.predict(X_test)
###########################

saveFile(classinfo_arr[svm_predict][:,1], test_video_id, "svm_video")
print("\t{:3.2f}s\n\n".format(time.time()-start))
print(f"train: {np.mean(cv_results['train_score'])}, "+\
      f"test: {np.mean(cv_results['test_score'])}")

###########################



SVM video descriptor
	8.81s


train: 0.6550742574257425, test: 0.12178217821782178
